## Ali's code

It seems that the way this link computes the weights of each topic in each document is not the optimal way (as it will not give you a weight if it is very small, but in big models, these small weights can also become important). Therefore instead, you can follow the following simple approach to get the weights:
To get the weights of a topic in the ith document, you call the method get_document_topics of the topic model object and pass the ith element of M1 (or tfidf_M1 in case you use tfidf) to it in the following way (minimum_probability=0 should be set so that none of near to zero entries are not removed) : 
lda_model.get_document_topics(M1[i],minimum_probability=0)
So let us see it in a complete example we had before: 

which gives the output (topics and weights of words in topics):

In [1]:
corpus = ["cryptography can be used for preventing data leakage in computer security",
"supervised learning and unsupervised learning are the two main groups of methods in machine learning",
"while in supervised learning we have access to the target variable in unsupervised learning we do not have such a variable",
"there are some methods in security for reducing the risk of information leakage like authentication and cryptography",
"topic modeling in an unsupervised machine learning model and therefore we do not have target variables"
]
stop_words = ["can","be","for","two","the","for","we","in","not","do","are","to","an","there","some","have","a","and","of","like","while","therefore","such"]

def clean_doc(doc):
    return " ".join([word for word in doc.lower().split() if word not in stop_words])

corpus_clean = [clean_doc(doc) for doc in corpus]
corpus_clean = [doc.split() for doc in corpus_clean]
from gensim import corpora
dictionary = corpora.Dictionary(corpus_clean)
dictionary.doc2bow(corpus_clean[2])
M1 = [dictionary.doc2bow(doc) for doc in corpus_clean]
import gensim
Lda = gensim.models.ldamodel.LdaModel
lda_model = Lda(M1, num_topics=2, id2word = dictionary, passes=5,random_state =0)
topics = lda_model.print_topics(num_topics=5, num_words=10)
for topic in topics:
    print(topic)

(0, '0.084*"learning" + 0.071*"cryptography" + 0.071*"security" + 0.071*"leakage" + 0.070*"methods" + 0.043*"computer" + 0.043*"data" + 0.043*"used" + 0.042*"preventing" + 0.042*"authentication"')
(1, '0.136*"learning" + 0.093*"unsupervised" + 0.082*"target" + 0.080*"variable" + 0.058*"machine" + 0.057*"supervised" + 0.050*"topic" + 0.050*"variables" + 0.050*"modeling" + 0.050*"model"')


Now, the following call gives us the weight of the two topics in the first document:

In [2]:
lda_model.get_document_topics(M1[0],minimum_probability=0)

[(0, 0.9348568), (1, 0.06514323)]

And here is a complete working code with tfidf :
Note that as we expect, the weight should be different here as the model has been changed. 


In [6]:
corpus = ["cryptography can be used for preventing data leakage in computer security",
"supervised learning and unsupervised learning are the two main groups of methods in machine learning",
"while in supervised learning we have access to the target variable in unsupervised learning we do not have such a variable",
"there are some methods in security for reducing the risk of information leakage like authentication and cryptography",
"topic modeling in an unsupervised machine learning model and therefore we do not have target variables"
]
stop_words = ["can","be","for","two","the","for","we","in","not","do","are","to","an","there","some","have","a","and","of","like","while","therefore","such"]

def clean_doc(doc):
    return " ".join([word for word in doc.lower().split() if word not in stop_words])

corpus_clean = [clean_doc(doc) for doc in corpus]
corpus_clean = [doc.split() for doc in corpus_clean]
from gensim import corpora
dictionary = corpora.Dictionary(corpus_clean)
dictionary.doc2bow(corpus_clean[2])
M1 = [dictionary.doc2bow(doc) for doc in corpus_clean]
import gensim
tfidf_model = gensim.models.TfidfModel(M1)
tfidf_M1 = tfidf_model[M1]
Lda = gensim.models.ldamodel.LdaModel
lda_model = Lda(tfidf_M1, num_topics=2, id2word = dictionary, passes=5,random_state =0)
topics = lda_model.print_topics(num_topics=5, num_words=10)
for topic in topics:
    print(topic)
lda_model.get_document_topics(tfidf_M1[0],minimum_probability=0)

(0, '0.058*"variable" + 0.055*"learning" + 0.046*"methods" + 0.045*"main" + 0.045*"cryptography" + 0.045*"groups" + 0.045*"security" + 0.045*"leakage" + 0.045*"supervised" + 0.043*"computer"')
(1, '0.060*"topic" + 0.059*"variables" + 0.059*"modeling" + 0.059*"model" + 0.050*"target" + 0.049*"machine" + 0.043*"learning" + 0.042*"unsupervised" + 0.035*"variable" + 0.035*"supervised"')


[(0, 0.83430034), (1, 0.16569969)]

# Our code

In [60]:
def readFile(filename):
    fileData = ""
    file = open("abstracts/" + filename, 'r')
    fileData = file.read().splitlines()
    file.close
    return fileData

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [61]:
import nltk
import gensim
from gensim import corpora
#nltk.download('stopwords')      # If this code block complains, comment out this line
from nltk.corpus import stopwords



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [62]:
def clean_doc(doc):
    return " ".join([word for word in doc.lower().split() if word not in stop_words])

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
corpus = []
abstracts
for i in range (2012,2020):
    conference = readFile('usenix' + str(i) + 'Abstracts.txt')
    for abstract in conference:
        corpus.append(abstract)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [64]:
stop_words = stopwords.words('english')

corpus_clean = [clean_doc(doc) for doc in corpus]
corpus_clean = [doc.split() for doc in corpus_clean]


dictionary = corpora.Dictionary(corpus_clean)
dictionary.doc2bow(corpus_clean[2])
M1 = [dictionary.doc2bow(doc) for doc in corpus_clean]

tfidf_model = gensim.models.TfidfModel(M1)
tfidf_M1 = tfidf_model[M1]



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
Lda = gensim.models.ldamodel.LdaModel
lda_model = Lda(tfidf_M1, num_topics=4, id2word = dictionary, passes=5, random_state =0)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### All topics

In [66]:
topics = lda_model.print_topics(num_topics=5, num_words=10)

for topic in topics:
    print(topic)


(0, '0.001*"attacks" + 0.000*"attack" + 0.000*"security" + 0.000*"data" + 0.000*"cache" + 0.000*"memory" + 0.000*"privacy" + 0.000*"system" + 0.000*"user" + 0.000*"network"')
(1, '0.001*"security" + 0.001*"attacks" + 0.001*"memory" + 0.000*"apps" + 0.000*"web" + 0.000*"network" + 0.000*"attack" + 0.000*"data" + 0.000*"analysis" + 0.000*"code"')
(2, '0.001*"malware" + 0.001*"security" + 0.001*"privacy" + 0.000*"password" + 0.000*"network" + 0.000*"data" + 0.000*"use" + 0.000*"devices" + 0.000*"students" + 0.000*"users"')
(3, '0.001*"data" + 0.001*"attacks" + 0.001*"security" + 0.000*"users" + 0.000*"web" + 0.000*"code" + 0.000*"user" + 0.000*"vulnerabilities" + 0.000*"system" + 0.000*"attack"')


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Visualisation of topics

In [67]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, tfidf_M1, dictionary)
vis

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.006738  0.009926       1        1  27.126679
2     -0.011110  0.002268       2        1  26.870972
3      0.005537 -0.008339       3        1  25.097243
0     -0.001165 -0.003856       4        1  20.905106, topic_info=             Term      Freq     Total Category  logprob  loglift
3342     students  1.000000  1.000000  Default  30.0000  30.0000
6021          usb  1.000000  1.000000  Default  29.0000  29.0000
1842      malware  3.000000  3.000000  Default  28.0000  28.0000
5147          cfi  1.000000  1.000000  Default  27.0000  27.0000
749         cache  1.000000  1.000000  Default  26.0000  26.0000
...           ...       ...       ...      ...      ...      ...
279      approach  0.498398  2.556681   Topic4  -8.1409  -0.0699
1061      control  0.489600  2.492943   Topic4  -8.1587  -0.0625
81          using  0.497620  2.838568   Topic4  -8.1425  -0.1760
269         users  0.495656  3.101261   Topic4  -8.1464  -0.2685
129   information  0.478061  2.848441   Topic4  -8.1826  -0.2196

[327 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
512       1  0.390062   access
512       2  0.390062   access
512       3  0.390062   access
95        2  0.498215  address
95        3  0.498215  address
...     ...       ...      ...
431       2  0.297849      web
431       3  0.297849      web
431       4  0.297849      web
1161      2  0.481671     work
1161      4  0.481671     work

[198 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 1])

### Topics per document

In [68]:
lda_model.get_document_topics(tfidf_M1[0],minimum_probability=0)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 0.9191931), (1, 0.02662838), (2, 0.026896829), (3, 0.027281724)]